## Convolution Layer
- 특정한 패턴의 특징이 어디서 나타나는지를 확인하는 도구
- 특징 자동 추출기

### 필터의 특징
1. 필터는 3차원 형태로  된 가중치의 모음
2. 필터셋 하나는 앞선 레이어의 결과인 "특징맵" 전체를 본다.
3. 필터셋 개수 만큼 특징맵을 만든다.

In [1]:
import tensorflow as tf
import pandas as pd

In [2]:
# 데이터를 준비
(independent, dependent), _ = tf.keras.datasets.mnist.load_data()
independent = independent.reshape(60000, 28, 28, 1)   # convolution연산은 3차원이여야 하므로
dependent = pd.get_dummies(dependent)   # onehot-encoding
print(independent.shape, dependent.shape)

11493376/11490434 [==============================] - 0s 0us/step
(60000, 28, 28, 1) (60000, 10)


In [4]:
# 모델을 만든다.
X = tf.keras.layers.Input(shape=[28,28,1])
H = tf.keras.layers.Conv2D(3, kernel_size=5, activation='swish')(X)   # 3개의 특징법 = 3채널의 특징법, 5x5사이즈
H = tf.keras.layers.Conv2D(6, kernel_size=5, activation='swish')(H)   # 6개의 특징법 = 6채널의 특징법, 5x5사이즈
H = tf.keras.layers.Flatten()(H)        # 한줄로 쭉 늘어뜨리다.
H = tf.keras.layers.Dense(84, activation='swish')(H)
Y = tf.keras.layers.Dense(10, activation='softmax')(H)
model = tf.keras.models.Model(X, Y)
model.compile(loss='categorical_crossentropy', metrics='accuracy')

In [5]:
# 모델을 학습
model.fit(independent, dependent, epochs=10)

Epoch 1/10
1875/1875 [==============================] - 12s 3ms/step - loss: 1.9433 - accuracy: 0.8630
Epoch 2/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0963 - accuracy: 0.9734
Epoch 3/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0636 - accuracy: 0.9827
Epoch 4/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0482 - accuracy: 0.9870
Epoch 5/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0379 - accuracy: 0.9899
Epoch 6/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0311 - accuracy: 0.9914
Epoch 7/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0279 - accuracy: 0.9919
Epoch 8/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0243 - accuracy: 0.9934
Epoch 9/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0228 - accuracy: 0.9935
Epoch 10/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.02

In [6]:
# 모델을 이용
pred = model.predict(independent[0:5])
pd.DataFrame(pred).round(2)

,0,1,2,3,4,5,6,7,8,9
0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [7]:
# 정답 확인
dependent[0:5]

,0,1,2,3,4,5,6,7,8,9
0,0,0,0,0,0,1,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,1


##### Conv2D를 통과하면 사이즈의 1만큼 줄어든다. 예를들면 kernel_size가 5이므로 (5,5,1)에서 1만큼 줄어 (4,4,1)을 (28,28,1)에서 빼준다. 그러면 결과는 (24,24,1)이 된다.

In [8]:
# 모델 확인
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 24, 24, 3)         78        
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 20, 20, 6)         456       
_________________________________________________________________
flatten (Flatten)            (None, 2400)              0         
_________________________________________________________________
dense (Dense)                (None, 84)                201684    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                850       
Total params: 203,068
Trainable params: 203,068
Non-trainable params: 0
_______________________________________________________

## Pooling Layer
- maxpooling


In [9]:
# convolution연산 했던것 그대로 사용
# 모델을 만드는 과정을 살짝 수정
# 모델을 만든다.
X = tf.keras.layers.Input(shape=[28,28,1])
H = tf.keras.layers.Conv2D(3, kernel_size=5, activation='swish')(X)   # 3개의 특징법 = 3채널의 특징법, 5x5사이즈
H = tf.keras.layers.MaxPool2D()(H)

H = tf.keras.layers.Conv2D(6, kernel_size=5, activation='swish')(H)   # 6개의 특징법 = 6채널의 특징법, 5x5사이즈
H = tf.keras.layers.MaxPool2D()(H)

H = tf.keras.layers.Flatten()(H)        # 한줄로 쭉 늘어뜨리다.
H = tf.keras.layers.Dense(84, activation='swish')(H)
Y = tf.keras.layers.Dense(10, activation='softmax')(H)
model = tf.keras.models.Model(X, Y)
model.compile(loss='categorical_crossentropy', metrics='accuracy')

In [10]:
# 모델을 학습
model.fit(independent, dependent, epochs=10)

Epoch 1/10
1875/1875 [==============================] - 6s 3ms/step - loss: 2.8683 - accuracy: 0.7637
Epoch 2/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1328 - accuracy: 0.9611
Epoch 3/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0955 - accuracy: 0.9718
Epoch 4/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0798 - accuracy: 0.9761
Epoch 5/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0723 - accuracy: 0.9781
Epoch 6/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0671 - accuracy: 0.9807
Epoch 7/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0621 - accuracy: 0.9814
Epoch 8/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0626 - accuracy: 0.9822
Epoch 9/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0643 - accuracy: 0.9819
Epoch 10/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.063

In [11]:
# 모델을 이용
pred = model.predict(independent[0:5])
pd.DataFrame(pred).round(2)

,0,1,2,3,4,5,6,7,8,9
0,0.0,0.0,0.0,0.3,0.0,0.7,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [12]:
# 정답 확인
dependent[0:5]

,0,1,2,3,4,5,6,7,8,9
0,0,0,0,0,0,1,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,1


In [13]:
# 모델 확인
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 3)         78        
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 3)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 8, 8, 6)           456       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 4, 4, 6)           0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 96)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 84)                8148